In [1]:
import unsloth
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/vinhpt/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/data/datnt3/text-normalization/core/model/saved/lora/vn-llama3.2-1b-finetuned-300k", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"role": "system",
#     "content": "You are a phonetic Vietnamese specialist mastering in Text normalization task in Text To Speech. Convert each numerical non-standard word into its spoken phonetic Vietnamese form, integrating it back into the sentence."},
#     {"role": "user", "content": """Convert the NSWs in the below sentence to spoken words: "đáng chú ý, tiến có 3 tiền án về tội trộm cắp tài sản." """}
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512,
#                    use_cache = True, temperature = 1.5, top_p = 0.9)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.762 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [119]:
messages = [
    {"role": "system",
    "content": """
    You are a phonetic Vietnamese specialist mastering in Text normalization task in Text To Speech. Convert each numerical non-standard word into its spoken phonetic Vietnamese form, integrating it back into the sentence.
    """},
    {"role": "user", "content": """
    Expand the below sentence having non-standard words into its spoken phonetic Vietnamese form. 
    
    "trước đó, lãnh đạo bộ thông tin và truyền thông đã chỉ đạo viettel và vnpt gấp rút xử lý việc báo điện tử vov bị tấn công mạng trong 2 ngày 12-13/6/2021.
    """}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True, temperature = 1.5, top_p = 0.9)

"trước đó, lãnh đạo bộ thông 

tin và truyền thông đã chỉ đạo viettel và vnpt gấp rút xử lý việc báo điện tử vov bị tấn công mạng trong hai ngày từ ngày mười hai đến ngày mười ba tháng sáu năm hai nghìn không trăm hai mươi mốt.<|eot_id|>


In [78]:
from evaluate import load
exact_match_metric = load("exact_match")

refs = ["đối với gói hỗ trợ theo nghị quyết một trăm mười sáu nq-cp, đến nay cơ bản nghệ an đã hoàn thành việc xác định số lượng giảm đóng cho sáu nghìn chín trăm bốn mươi ba đơn vị, với một trăm năm mươi tám nghìn chín trăm mười một lao động, kinh phí tạm tính được điều chỉnh (từ tháng mười năm hai nghìn hai mươi mốt đến tháng chín năm hai nghìn hai mươi hai) khoảng tám mươi lăm phẩy sáu tỷ đồng."]
preds = ["đối với gói hỗ trợ theo nghị quyết một trăm mười sáu /nq-cp, đến nay cơ bản nghệ an đã hoàn thành việc xác định số lượng giảm đóng cho sáu nghìn chín trăm bốn mươi ba đơn vị, với một trăm năm mươi tám nghìn chín trăm mười một lao động, kinh phí tạm tính được điều chỉnh (từ tháng mười năm hai nghìn không trăm hai mươi mốt đến tháng chín năm hai nghìn không trăm hai mươi hai) khoảng tám mươi lăm phẩy sáu tỷ đồng."]

results = exact_match_metric.compute(references=refs, predictions=preds, regexes_to_ignore=[" không trăm","/"], ignore_case=True, ignore_punctuation=True)
results

{'exact_match': np.float64(1.0)}